## Full Run

In [1]:
import os
os.chdir('..')

In [2]:
Xtrain_dir = 'solar/data/kaggle_solar/train/'
Xtest_dir = 'solar/data/kaggle_solar/test'
ytrain_file = 'solar/data/kaggle_solar/train.csv'
station_file = 'solar/data/kaggle_solar/station_info.csv'
import solar.wrangle.wrangle
import solar.wrangle.subset
import solar.wrangle.engineer
import solar.analyze.model
import solar.report.submission
import numpy as np

In [3]:
# Choose up to 98 stations; not specifying a station means to use all that fall within the given lats and longs. If the
# parameter 'all' is given, then it will use all stations no matter the provided lats and longs
station = ['all']

# Determine which dates will be used to train the model. No specified date means use the entire set from 1994-01-01 
# until 2007-12-31.
train_dates = ['1994-01-01', '2007-12-31']

#2008-01-01 until 2012-11-30
test_dates = ['2008-01-01', '2012-11-30']

station_layout = True

# Use all variables
var = ['all']

# Keep model 0 (the default model) as a column for each of the variables (aggregated over other dimensions)
model = [0]

# Aggregate over all times
times = ['all']

default_grid = {'type':'relative', 'axes':{'var':var, 'models':model, 'times':times,
                                          'station':station}}
# This just uses the station_names as another feature
stat_names = {'type':'station_names'}

grid_and_stat_feats = [stat_names, default_grid]


In [4]:
import solar.report.submission
import solar.wrangle.wrangle
import solar.wrangle.subset
import solar.wrangle.engineer
import solar.analyze.model

In [5]:
# test combination of station names and grid
reload(solar.wrangle.wrangle)
reload(solar.wrangle.subset)
reload(solar.wrangle.engineer)
from solar.wrangle.wrangle import SolarData

input_data = SolarData.load(Xtrain_dir, ytrain_file, Xtest_dir, station_file, \
                                  train_dates, test_dates, station, \
                                  station_layout, grid_and_stat_feats)

['ACME' 'ADAX' 'ALTU' 'APAC' 'ARNE' 'BEAV' 'BESS' 'BIXB' 'BLAC' 'BOIS'
 'BOWL' 'BREC' 'BRIS' 'BUFF' 'BURB' 'BURN' 'BUTL' 'BYAR' 'CAMA' 'CENT'
 'CHAN' 'CHER' 'CHEY' 'CHIC' 'CLAY' 'CLOU' 'COOK' 'COPA' 'DURA' 'ELRE'
 'ERIC' 'EUFA' 'FAIR' 'FORA' 'FREE' 'FTCB' 'GOOD' 'GUTH' 'HASK' 'HINT'
 'HOBA' 'HOLL' 'HOOK' 'HUGO' 'IDAB' 'JAYX' 'KENT' 'KETC' 'LAHO' 'LANE'
 'MADI' 'MANG' 'MARE' 'MAYR' 'MCAL' 'MEDF' 'MEDI' 'MIAM' 'MINC' 'MTHE'
 'NEWK' 'NINN' 'NOWA' 'OILT' 'OKEM' 'OKMU' 'PAUL' 'PAWN' 'PERK' 'PRYO'
 'PUTN' 'REDR' 'RETR' 'RING' 'SALL' 'SEIL' 'SHAW' 'SKIA' 'SLAP' 'SPEN'
 'STIG' 'STIL' 'STUA' 'SULP' 'TAHL' 'TALI' 'TIPT' 'TISH' 'VINI' 'WASH'
 'WATO' 'WAUR' 'WEAT' 'WEST' 'WILB' 'WIST' 'WOOD' 'WYNO']
98


KeyboardInterrupt: 

In [ ]:
reload(solar.analyze.model)
import numpy as np
from solar.analyze.model import Model

from sklearn.linear_model import Ridge
from sklearn import metrics

error_formula = 'mean_absolute_error'

model = Model.model(input_data, Ridge, {'alpha':np.logspace(-3,1,8,base=10)}, 10, 
                    error_formula, normalize=True)

In [ ]:
reload(solar.report.submission)
from solar.report.submission import Submission

preds = Submission.make_submission_file(model, input_data[1], input_data[2], {'grid'})